# General How-To
-  there are 6 total pdfs, 4 are easy and 2 are hard & unstructured

steps for each pdf
1. pdf to images for each page
2. how to choose the image page we want?
3. from there, how to choose only the part of the page we want?
4. (repeat steps for all pdfs -- to make it easier, put all the above 3 steps in functions)

ideas
- pdftoimage or any other libraries possible?
- regex to look for 'test phase'

## Will be using confusion matrix throughout this to quantify goodness of results

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

#### just manually find out which pages have table (any table) & useful table (have test phase tables)

In [ ]:
y_test_table = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [ ]:
y_test_useful_table = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0,0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1]

#### create function to make path for easy function calls later on

In [ ]:
def make_path(page):
    return 'Data/P00296_B014.pdf-page'+ str(page)+'.jpg'
page_numbers = np.arange(42)
all_paths = list(map(make_path,page_numbers))
all_paths

['Data/P00296_B014.pdf-page0.jpg',
 'Data/P00296_B014.pdf-page1.jpg',
 'Data/P00296_B014.pdf-page2.jpg',
 'Data/P00296_B014.pdf-page3.jpg',
 'Data/P00296_B014.pdf-page4.jpg',
 'Data/P00296_B014.pdf-page5.jpg',
 'Data/P00296_B014.pdf-page6.jpg',
 'Data/P00296_B014.pdf-page7.jpg',
 'Data/P00296_B014.pdf-page8.jpg',
 'Data/P00296_B014.pdf-page9.jpg',
 'Data/P00296_B014.pdf-page10.jpg',
 'Data/P00296_B014.pdf-page11.jpg',
 'Data/P00296_B014.pdf-page12.jpg',
 'Data/P00296_B014.pdf-page13.jpg',
 'Data/P00296_B014.pdf-page14.jpg',
 'Data/P00296_B014.pdf-page15.jpg',
 'Data/P00296_B014.pdf-page16.jpg',
 'Data/P00296_B014.pdf-page17.jpg',
 'Data/P00296_B014.pdf-page18.jpg',
 'Data/P00296_B014.pdf-page19.jpg',
 'Data/P00296_B014.pdf-page20.jpg',
 'Data/P00296_B014.pdf-page21.jpg',
 'Data/P00296_B014.pdf-page22.jpg',
 'Data/P00296_B014.pdf-page23.jpg',
 'Data/P00296_B014.pdf-page24.jpg',
 'Data/P00296_B014.pdf-page25.jpg',
 'Data/P00296_B014.pdf-page26.jpg',
 'Data/P00296_B014.pdf-page27.jpg',
 '

# Code - on 1 pdf (P00296_B014.pdf)

In [ ]:
pip install pdf2image

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import numpy as np
import cv2
from pdf2image import convert_from_path

## Step 1) PDF to image for each page

input is pdf, output is image for each page

In [ ]:
pdf_path = 'Data/P00296_B014.pdf'
images = convert_from_path(pdf_path, 500)

for i in range(len(images)):
      # Save pages as images in the pdf
    images[i].save(pdf_path +'-page'+ str(i) +'.jpg', 'JPEG')

now have all 43 pages of P00296_B014.pdf in 'Data' folder

## Step 2) Choosing the page we want

### try each method for 1 page that have test phase ('P00296_B014.pdf-page11.jpg') and 1 that dont have ('P00296_B014.pdf-page1.jpg')

what we need to get
- tables we need are in: page 12-14 (11-13 in data bc 0 index), 16-18 (15-17), 26-30 (25-29), 32-36 (31-35), 38-42 (37-41)
- if get all the tables, should get # of tables: 6 + 6 + 6 + 6 + 6 = 30 tables!

general flow
- input is image, output is page number we need? (e.g. list: ['P00296_B014.pdf-page11.jpg', 'P00296_B014.pdf-page12.jpg' ....]
- trying it for 1 page that have test phase ('P00296_B014.pdf-page11.jpg') and 1 that dont have ('P00296_B014.pdf-page1.jpg')

brainstorm how to get
- regex to detect using all have the "TEST PHASE : (FLOW/SHUTIN PERIOD) # (number)" above the table
- Look for specific plots or columns
- Display every page in jupyter or sth condensed -> take a quick look
    - E.g. display each image in a thumbnail -> user manually pick -> manually pick and it will filter automatically
- Pattern matching; e.g. every 2 pages have a table we need, always have 30 tables in each pdf
- How much black the image has on it = how much text -> can indicate a lot of info on that page = prob a table?

problems i foresee
- some tables are split into diff pages (e.g. table for test phase 1 flow period #3 is between page 17 and 18)

### Method 1: Pytesseract + Regex

In [ ]:
## have test phase ('P00296_B014.pdf-page11.jpg')

from PIL import Image
import pytesseract

# Load the image
image = Image.open('Data/P00296_B014.pdf-page11.jpg')  # Replace 'your_image.png' with the path to your image file

# Convert the image to grayscale (optional but can help simplify the image)
gray_image = image.convert('L')

# Define the phrase you want to check for
desired_phrase = ["TEST PHASE", "FLOW PERIOD", "SHUTIN PERIOD"]

# Iterate through the image pixel by pixel
for y in range(image.height - 1):
    for x in range(image.width - len(desired_phrase)):
        crop = gray_image.crop((x, y, x + len(desired_phrase), y + 1))
        crop_text = pytesseract.image_to_string(crop)
        if crop_text.strip() == desired_phrase:
            print(f"The image contains the phrase: {desired_phrase}")
            break

print(f"The image does not contain the phrase: {desired_phrase}")

In [ ]:
## dont have ('P00296_B014.pdf-page1.jpg')

#### for some reason, this took forever to run

### Method 2: How much black the image has on it = how much text -> more black = more info = likely a table

In [ ]:
def calculate_black_percentage(image_path):
    # Read the image
    img = cv2.imread(image_path)

    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Threshold the image to get a binary image with black and white pixels
    _, thresholded = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY)

    # Calculate the number of black pixels
    num_black_pixels = np.sum(thresholded == 0)

    # Calculate the total number of pixels in the image
    total_pixels = img.shape[0] * img.shape[1]

    # Calculate the percentage of black pixels
    percentage_black = (num_black_pixels / total_pixels) * 100
    return percentage_black

In [ ]:
def black_perc_table(image_path):
    percentage_black = calculate_black_percentage(image_path)
    if percentage_black > 0.5:
        return 1
    else:
        return 0

have table ('P00296_B014.pdf-page11.jpg')

In [ ]:
black_percentage = calculate_black_percentage('Data/P00296_B014.pdf-page11.jpg')
print(f"The percentage of black in the image is: {black_percentage:.2f}%")

The percentage of black in the image is: 0.52%


dont have ('P00296_B014.pdf-page1.jpg')

In [ ]:
black_percentage = calculate_black_percentage('Data/P00296_B014.pdf-page1.jpg')
print(f"The percentage of black in the image is: {black_percentage:.2f}%")

The percentage of black in the image is: 0.60%


#### Conclusion: page that have table has less black % = cannot use this method

In [ ]:
y_pred_black = list(map(black_perc_table, all_paths))
print(confusion_matrix(y_test_table, y_pred_black))
print(classification_report(y_test_table,y_pred_black))

[[11  5]
 [18  8]]
              precision    recall  f1-score   support

           0       0.38      0.69      0.49        16
           1       0.62      0.31      0.41        26

    accuracy                           0.45        42
   macro avg       0.50      0.50      0.45        42
weighted avg       0.53      0.45      0.44        42



### Method 3: Use Canny Edge Detection to detect edges in the image -> well-defined edges = table

In [ ]:
def detect_table(image_path):
    # Read the image
    img = cv2.imread(image_path)

    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply GaussianBlur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Apply Canny edge detection
    edges = cv2.Canny(blurred, 50, 150)

    # Calculate the number of edges detected
    num_edges = cv2.countNonZero(edges)

    # Define a threshold for edge count to determine if there might be a table
    edge_threshold = 5000  # You might need to adjust this threshold based on your images

    # Determine if a table is likely present based on edge count
    has_table = num_edges > edge_threshold

    return has_table


In [ ]:
result = detect_table('Data/P00296_B014.pdf-page11.jpg')
result

True

In [ ]:
# this image correct output: no table, but it is a plot = may confuse the alg
result = detect_table('Data/P00296_B014.pdf-page1.jpg')
result

True

In [ ]:
y_pred_canny = list(map(detect_table, all_paths))
print(confusion_matrix(y_test_table, y_pred_canny))
print(classification_report(y_test_table,y_pred_canny))

[[ 0 16]
 [ 0 26]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        16
           1       0.62      1.00      0.76        26

    accuracy                           0.62        42
   macro avg       0.31      0.50      0.38        42
weighted avg       0.38      0.62      0.47        42



/Users/sandy/opt/anaconda3/envs/BOEMddiscovery/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sandy/opt/anaconda3/envs/BOEMddiscovery/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sandy/opt/anaconda3/envs/BOEMddiscovery/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf

### Method 4: Pattern matching -- e.g. every 2 pages have a table we need, always have 30 tables in each pdf

tables we need are in: page 12-14 (11-13 in data bc 0 index), 16-18 (15-17), 26-30 (25-29), 32-36 (31-35), 38-42 (37-41)
if get all the tables, should get # of tables: 6 + 6 + 6 + 6 + 6 = 30 tables!

In [ ]:
# find where first table appears -> page11

# from there, have 6 tables in each set (in case of pg 11, these 6 tables spread over pg 11 12 13)

# skip to next set
## how to know where next set is? hmm

# get total 30 tables

### Method 5: Display every page in jupyter or sth condensed -> take a quick look
E.g. display each image in a thumbnail -> user manually pick -> manually pick and it will filter automatically

### Method 6: Using pre-trained deep learning model: YOLO

## Step 3) Filter to get only pages with wanted tables

#### go into this step assuming already have list of images that FOR SURE have tables -> expected output should be > 30 tables

In [ ]:
pages_with_any_table = [10, 11,12,13, 14,15,16,17, 24,25,26,27,28,29, 30,31,32,33,34,35, 36,37,38,39,40,41]

any_table_paths = list(map(make_path, pages_with_any_table))
any_table_paths

['Data/P00296_B014.pdf-page10.jpg',
 'Data/P00296_B014.pdf-page11.jpg',
 'Data/P00296_B014.pdf-page12.jpg',
 'Data/P00296_B014.pdf-page13.jpg',
 'Data/P00296_B014.pdf-page14.jpg',
 'Data/P00296_B014.pdf-page15.jpg',
 'Data/P00296_B014.pdf-page16.jpg',
 'Data/P00296_B014.pdf-page17.jpg',
 'Data/P00296_B014.pdf-page24.jpg',
 'Data/P00296_B014.pdf-page25.jpg',
 'Data/P00296_B014.pdf-page26.jpg',
 'Data/P00296_B014.pdf-page27.jpg',
 'Data/P00296_B014.pdf-page28.jpg',
 'Data/P00296_B014.pdf-page29.jpg',
 'Data/P00296_B014.pdf-page30.jpg',
 'Data/P00296_B014.pdf-page31.jpg',
 'Data/P00296_B014.pdf-page32.jpg',
 'Data/P00296_B014.pdf-page33.jpg',
 'Data/P00296_B014.pdf-page34.jpg',
 'Data/P00296_B014.pdf-page35.jpg',
 'Data/P00296_B014.pdf-page36.jpg',
 'Data/P00296_B014.pdf-page37.jpg',
 'Data/P00296_B014.pdf-page38.jpg',
 'Data/P00296_B014.pdf-page39.jpg',
 'Data/P00296_B014.pdf-page40.jpg',
 'Data/P00296_B014.pdf-page41.jpg']

#### this is an example of a page with a table, but not the kind we want

need to get rid of 10,14,24,30,36 -- they're all formatted the below way

### To approach this problem, can either a) filter IN wanted pdfs (with 'test phase' on it), or b) filter OUT uwanted pdfs (in this case, with 'well test data printout' on it)

we will be using the a) filtering IN wanted pdfs because doing b) filtering out will need a different condition for each different pdf; each pdf has excess tables with different indicators

### Method 1: Regex

In [ ]:
pattern = "\\s*TEST PHASE\\s:*"


### Method 2: OCR

In [ ]:
def detect_text_in_image(image_path):
    img = Image.open(image_path)
    text = pytesseract.image_to_string(img)
    if "TEST PHASE" in text:
        return True
    else:
        return False

In [ ]:
expected_no = detect_text_in_image('Data/P00296_B014.pdf-page10.jpg')

expected_yes_easy = detect_text_in_image('Data/P00296_B014.pdf-page11.jpg')
expected_yes_hard = detect_text_in_image('Data/P00296_B014.pdf-page38.jpg')
print(expected_no, expected_yes_easy, expected_yes_hard)

False True False


seen above: simple OCR where immediately image_to_string works for no cases, but only for EASY yes cases (high quality, no smudges)

In [ ]:
y_pred_useful_ocr = list(map(detect_text_in_image, all_paths))
print(confusion_matrix(y_test_useful_table, y_pred_useful_ocr))
print(classification_report(y_test_useful_table,y_pred_useful_ocr))

[[21  0]
 [ 7 14]]
              precision    recall  f1-score   support

           0       0.75      1.00      0.86        21
           1       1.00      0.67      0.80        21

    accuracy                           0.83        42
   macro avg       0.88      0.83      0.83        42
weighted avg       0.88      0.83      0.83        42



### Method 3: Template Matching

In [ ]:
def template_matching(image_path, template_path='Data/P00296_B014.pdf-page11.jpg', threshold=0.1):
    target_image = cv2.imread(image_path)
    template = cv2.imread(template_path)

    # Ensure the template image is smaller or equal to the target image
    target_height, target_width = target_image.shape[:2]
    template = cv2.resize(template, (target_width, target_height))

    target_gray = cv2.cvtColor(target_image, cv2.COLOR_BGR2GRAY)
    template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

    result = cv2.matchTemplate(target_gray, template_gray, cv2.TM_CCOEFF_NORMED)

    # Finding the maximum match value and its location
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)

    if max_val >= threshold:
        return True
    else:
        return False

In [ ]:
template = 'Data/P00296_B014.pdf-page11.jpg'

expected_no = template_matching('Data/P00296_B014.pdf-page10.jpg', template, threshold=0.1)
expected_yes_easy = template_matching('Data/P00296_B014.pdf-page12.jpg', template, threshold=0.1)
expected_yes_easy = template_matching('Data/P00296_B014.pdf-page38.jpg', template, threshold=0.1)
print(expected_no, expected_yes_easy, expected_yes_easy)

False True True


In [ ]:
y_pred_useful_tm = list(map(template_matching, all_paths))
print(confusion_matrix(y_test_useful_table, y_pred_useful_tm))
print(classification_report(y_test_useful_table,y_pred_useful_tm))

[[ 4 17]
 [ 2 19]]
              precision    recall  f1-score   support

           0       0.67      0.19      0.30        21
           1       0.53      0.90      0.67        21

    accuracy                           0.55        42
   macro avg       0.60      0.55      0.48        42
weighted avg       0.60      0.55      0.48        42



seen above: even the easy yes only become yes with a fairly threshold of 0.4 -- low because only similarity is 'test phase' and header names, all the numbers in the table are different

### Method 4: Manual Pattern Matching

in this case, see that the unwanted pages are always the first number in each set (e.g. page 10 in 10,11,12,13) = can just get rid of the smallest number in each consecutive set

In [ ]:
# separate list into sets where each set is consecutive numbers
# then get rid of the first number in each set

#### we now have filtered the list down to only pages that have tables that have 'TEST PHASE' on it

In [ ]:
pages_with_wanted_tables = [11,12,13, 15,16,17, 25,26,27,28,29, 31,32,33,34,35, 37,38,39,40,41]

wanted_table_paths = list(map(make_path, pages_with_wanted_tables))
wanted_table_paths

['Data/P00296_B014.pdf-page11.jpg',
 'Data/P00296_B014.pdf-page12.jpg',
 'Data/P00296_B014.pdf-page13.jpg',
 'Data/P00296_B014.pdf-page15.jpg',
 'Data/P00296_B014.pdf-page16.jpg',
 'Data/P00296_B014.pdf-page17.jpg',
 'Data/P00296_B014.pdf-page25.jpg',
 'Data/P00296_B014.pdf-page26.jpg',
 'Data/P00296_B014.pdf-page27.jpg',
 'Data/P00296_B014.pdf-page28.jpg',
 'Data/P00296_B014.pdf-page29.jpg',
 'Data/P00296_B014.pdf-page31.jpg',
 'Data/P00296_B014.pdf-page32.jpg',
 'Data/P00296_B014.pdf-page33.jpg',
 'Data/P00296_B014.pdf-page34.jpg',
 'Data/P00296_B014.pdf-page35.jpg',
 'Data/P00296_B014.pdf-page37.jpg',
 'Data/P00296_B014.pdf-page38.jpg',
 'Data/P00296_B014.pdf-page39.jpg',
 'Data/P00296_B014.pdf-page40.jpg',
 'Data/P00296_B014.pdf-page41.jpg']

## Now, use this list of page paths in Preprocess + OCR next stage